# Task Solving with Provided Tools as Functions


* In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to make function calls with the new feature of latest models like OpenAI / Llama3, etc.

* A specified prompt and function configs must be passed to `AssistantAgent` to initialize the agent. 

* The corresponding functions must be passed to `UserProxyAgent`, which will execute any function calls made by `AssistantAgent`.

* Besides this requirement of matching descriptions with functions, we recommend checking the system message in the `AssistantAgent` to ensure the instructions align with the function call descriptions.


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install `pyautogen`:
```bash
pip install pyautogen
```

In [1]:
# %pip install "pyautogen>=0.2.3"

### Imports

In [2]:
#from typing import Literal
from typing_extensions import Annotated
import autogen
from autogen.cache import Cache
from Tools_r2 import calculate, currency_converter, ddg_search, get_news

#config_list = autogen.config_list_from_json(
#    "OAI_CONFIG_LIST",
#    filter_dict={"tags": ["3.5-tool"]},  # comment out to get all
#)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


### LLM Configuration setting

In [3]:
import os
config_list = [{"api_type":"groq","model":"llama3-70b-8192","api_key":os.environ.get("GROQ_API_KEY"),},
               {"api_type":"groq","model":"llama3-8b-8192","api_key":os.environ.get("GROQ_API_KEY"),}]

llm_config = {
    "config_list": config_list,
    "timeout": 120,
}

## Creating Agents

In [4]:
# Creating chatbot
Assistant_Agent = autogen.AssistantAgent(
    name="assistant_agent",
    system_message="""You are an intelligent agent capable of answering questions and performing tasks, like 
    For currency exchange tasks, web search tasks and news search tasks when required. only use the functions you have been provided with.
    Reply TERMINATE when the task is done.""",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy" 
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
)

### Tools Functions (converting to autogen recommended format)

In [5]:
def currency_converter_ag(
    amount: Annotated[str, "Amount of currency in base_currency"],
    base_currency: Annotated[str, "Base_currency symbol"] = "USD",
    quote_currency: Annotated[str, "Quote currency symbol"] = "GBP",
) -> str:
    result = currency_converter(amount, base_currency, quote_currency)
    return result

def ddg_search_ag(
    topic: Annotated[str, "Topic to search for news"]="world news",
    num_headlines: Annotated[int, "Number of headlines to return"] = 5,
    )-> str:
    search_results = ddg_search(topic, num_headlines)
    return search_results
 
def get_news_ag(
    topic: Annotated[str, "Topic to search for news"]="world news",
    num_headlines: Annotated[int, "Number of headlines to return"] = 5,
    )-> str:
    news_results = get_news(topic, num_headlines)
    return news_results

### Registering the tools with Agents

In [6]:
autogen.agentchat.register_function(
    currency_converter_ag,
    caller=Assistant_Agent,
    executor=user_proxy,
    description="Currency exchange calculator.",
)

autogen.agentchat.register_function(
    ddg_search_ag,
    caller=Assistant_Agent,
    executor=user_proxy,
    description="Search on DuckDuckGo.",
)

autogen.agentchat.register_function(
    get_news_ag,
    caller=Assistant_Agent,
    executor=user_proxy,
    description="Get latest news.",
)

### Checking Tools availability with Assistant Agent

In [7]:
Assistant_Agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Currency exchange calculator.',
   'name': 'currency_converter_ag',
   'parameters': {'type': 'object',
    'properties': {'amount': {'type': 'string',
      'description': 'Amount of currency in base_currency'},
     'base_currency': {'type': 'string',
      'default': 'USD',
      'description': 'Base_currency symbol'},
     'quote_currency': {'type': 'string',
      'default': 'GBP',
      'description': 'Quote currency symbol'}},
    'required': ['amount']}}},
 {'type': 'function',
  'function': {'description': 'Search on DuckDuckGo.',
   'name': 'ddg_search_ag',
   'parameters': {'type': 'object',
    'properties': {'topic': {'type': 'string',
      'default': 'world news',
      'description': 'Topic to search for news'},
     'num_headlines': {'type': 'integer',
      'default': 5,
      'description': 'Number of headlines to return'}},
    'required': []}}},
 {'type': 'function',
  'function': {'description': 'Get latest news

### Sample 1 Query

In [8]:
with Cache.disk() as cache:
    # start the conversation
    query = """"What's are the top 10 latest news about AI (list in bullets)
and how much is 100 USD in EUR?"""

    res = user_proxy.initiate_chat(Assistant_Agent, message=query, summary_method="reflection_with_llm", cache=cache)

user_proxy (to assistant_agent):

"What's are the top 10 latest news about AI (list in bullets)
and how much is 100 USD in EUR?

--------------------------------------------------------------------------------
assistant_agent (to user_proxy):

***** Suggested tool call (call_apdp): get_news_ag *****
Arguments: 
{"num_headlines":10,"topic":"AI news"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_news_ag...
user_proxy (to assistant_agent):

user_proxy (to assistant_agent):

***** Response from calling tool (call_apdp) *****
[
  {
    "date": "2024-10-18T20:48:00+00:00",
    "title": "AI Chatbots Behind The Latest Memecoin Trend",
    "body": "Tokens created for Terminal of Truths, an AI millionaire chatbot, and Fi, a female AI character, may be the start of the latest trend among memecoins",
    "url": "https://www.forbes.com/sites/digital-assets/2024/10/18/ai-cha